In [8]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

In [9]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}
            
    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping
            

        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)
             
        return X_encoded 

In [10]:
syn_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043_set0.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.20571,0.000000,0.000000,1.401394,1.016979,-0.307006,1.016979,0.629113,-1.055231,-0.307006,-0.307006,0.629113,0.629113,-1.055231,0.000000,-0.307006,0.000000,0.629113,0.000000,-0.307006
1,0.20571,0.205710,-0.307006,0.629113,-0.307006,1.016979,0.205710,0.205710,-0.307006,1.016979,-1.055231,1.401394,0.629113,1.401394,0.205710,-0.307006,1.401394,1.016979,0.205710,-1.055231
2,0.20571,-1.055231,-1.055231,0.629113,0.205710,-0.307006,0.205710,-1.055231,-1.055231,0.205710,-0.307006,-1.055231,0.205710,1.401394,-0.307006,0.205710,0.000000,-0.307006,-1.055231,-1.055231
3,0.20571,-0.307006,-1.055231,1.401394,-1.055231,-1.055231,-0.307006,0.205710,0.205710,0.205710,0.000000,-1.055231,1.016979,0.629113,-1.055231,-0.307006,0.000000,-0.307006,0.000000,0.205710
4,0.20571,0.205710,-1.055231,-0.307006,0.205710,1.401394,1.016979,-0.307006,1.016979,0.205710,0.205710,0.205710,0.205710,0.629113,-0.307006,1.812683,0.205710,0.205710,0.000000,-0.307006


In [11]:
org_test_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.569715,-0.155969,1.586662,-0.110356,-0.021324,-0.412361,0.229132,-0.033252,-0.033252,-0.482741,0.699000,0.610469,0.630512,0.728991,0.268615,0.367391,0.733324,0.331370,0.885785,0.967358
1,-1.172108,-0.155969,-1.191184,-1.455578,0.023052,0.318783,0.229132,0.003738,0.169368,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,0.733324,0.331370,-0.692514,-0.506478
2,0.967358,-0.155969,-1.233862,-0.259956,-0.021324,-0.412361,-0.696401,0.003738,-0.141946,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-2.375949,-0.650735,-0.422533,0.569715
3,0.136166,-0.155969,1.586662,0.967358,-0.021324,0.318783,0.229132,0.003738,-0.141946,0.722046,0.699000,0.610469,-0.269859,0.728991,0.268615,0.367391,0.733324,-0.650735,0.885785,-0.390795
4,0.136166,-0.155969,-1.593819,1.363492,0.023052,0.318783,0.229132,0.003738,-0.141946,-0.482741,0.699000,0.610469,0.630512,-0.831894,0.268615,0.367391,-2.375949,-0.650735,-0.692514,0.136166


In [12]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1413 126 225 349
pod:  0.60801393728223
pof:  0.08187134502923976
AUC:  0.7630712961264952
recall:  0.60801393728223
precision:  0.7347368421052631
F1-Score:  0.665395614871306
accuracy:  0.8338854708944629


In [13]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1365 174 187 387
pod:  0.6742160278745645
pof:  0.11306042884990253
AUC:  0.7805777995123309
recall:  0.6742160278745645
precision:  0.6898395721925134
F1-Score:  0.6819383259911894
accuracy:  0.8291528632276385


In [14]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1384 155 249 325
pod:  0.5662020905923345
pof:  0.10071474983755685
AUC:  0.7327436703773889
recall:  0.5662020905923345
precision:  0.6770833333333334
F1-Score:  0.6166982922201139
accuracy:  0.8088026502602934


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1440 99 304 270
pod:  0.47038327526132406
pof:  0.06432748538011696
AUC:  0.7030278949406035
recall:  0.47038327526132406
precision:  0.7317073170731707
F1-Score:  0.5726405090137857
accuracy:  0.8092759110269758


In [16]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1377 162 304 270
pod:  0.47038327526132406
pof:  0.10526315789473684
AUC:  0.6825600586832936
recall:  0.47038327526132406
precision:  0.625
F1-Score:  0.5367793240556661
accuracy:  0.779460482725982


In [17]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1400 139 246 328
pod:  0.5714285714285714
pof:  0.0903183885640026
AUC:  0.7405550914322844
recall:  0.5714285714285714
precision:  0.702355460385439
F1-Score:  0.6301633045148896
accuracy:  0.8177946048272599


In [18]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0) 

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1387 152 236 338
pod:  0.5888501742160279
pof:  0.09876543209876543
AUC:  0.7450423710586312
recall:  0.5888501742160279
precision:  0.689795918367347
F1-Score:  0.6353383458646616
accuracy:  0.8163748225272125


In [19]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix
 
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)    

Epoch 1/30
124/124 [==============================] - 2s 7ms/step - loss: 0.2613 - accuracy: 0.8912 - val_loss: 0.2210 - val_accuracy: 0.9047
Epoch 2/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1890 - accuracy: 0.9262 - val_loss: 0.1921 - val_accuracy: 0.9229
Epoch 3/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1716 - accuracy: 0.9308 - val_loss: 0.1820 - val_accuracy: 0.9239
Epoch 4/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1597 - accuracy: 0.9359 - val_loss: 0.1747 - val_accuracy: 0.9310
Epoch 5/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1538 - accuracy: 0.9397 - val_loss: 0.1679 - val_accuracy: 0.9320
Epoch 6/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1433 - accuracy: 0.9422 - val_loss: 0.1550 - val_accuracy: 0.9402
Epoch 7/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1376 - accuracy: 0.9455 - val_loss: 0.1506 - val_accuracy: 0.9381
Epoch 